In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [2]:

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [3]:
cap = cv2.VideoCapture(0)
ALERT_DURATION = 15  
closed_eyes_counter = 0
threshold = 120 
alert_threshold = 15

In [4]:
model = load_model('drowsiness_model.h5')  

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(100, 100))


    focus_status = "Not Focused"
    eye_open = False

    for (fx, fy, fw, fh) in faces:

        face_color = (0, 255, 0) if eye_open else (0, 0, 255)
        cv2.rectangle(frame, (fx, fy), (fx + fw, fy + fh), face_color, 2)


        roi_gray = gray[fy:fy + fh, fx:fx + fw]
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=3)

        for (ex, ey, ew, eh) in eyes:
            if ew < 25 or eh < 25:
                continue

            eye_roi = roi_gray[ey:ey + eh, ex:ex + ew]
            eye_roi = cv2.resize(eye_roi, (48, 48))
            eye_roi = eye_roi.astype("float32") / 255.0
            eye_roi = np.expand_dims(eye_roi, axis=-1)
            eye_roi = np.expand_dims(eye_roi, axis=0)

            prediction = model.predict(eye_roi, verbose=0)[0]
            print(prediction)
            open_prob = prediction[1]

            if open_prob > 0.6:
                eye_open = True


    if eye_open:
        focus_status = "Focused"
        closed_eyes_counter = 0
    else:
        closed_eyes_counter += 1


    cv2.putText(frame, f"Focus: {focus_status}", (20, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0) if focus_status == "Focused" else (0, 0, 255), 2)


    if closed_eyes_counter >= ALERT_DURATION:
        cv2.putText(frame, "DROWSINESS ALERT!", (20, 100),  
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    cv2.imshow("Drowsiness Detector (Face + Eyes)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[1.6605592e-05 9.9998343e-01]
[2.0345063e-04 9.9979657e-01]
[2.1979510e-04 9.9978024e-01]
[4.6465665e-04 9.9953532e-01]
[2.8606589e-04 9.9971396e-01]
[2.6124237e-05 9.9997389e-01]
[7.089212e-05 9.999291e-01]
[2.2200214e-04 9.9977797e-01]
[5.719479e-05 9.999428e-01]
[1.5067469e-04 9.9984932e-01]
[1.09774235e-04 9.99890208e-01]
[2.2460773e-04 9.9977535e-01]
[1.2556973e-04 9.9987447e-01]
[2.9901907e-04 9.9970102e-01]
[5.2990312e-05 9.9994695e-01]
[3.7168633e-04 9.9962831e-01]
[0.02583263 0.9741674 ]
[8.0017475e-05 9.9992001e-01]
[0.00301481 0.99698514]
[0.13149233 0.8685077 ]
[7.2065376e-07 9.9999928e-01]
[3.850098e-06 9.999962e-01]
[0.01168686 0.98831314]
[1.1778253e-07 9.9999988e-01]
[2.8075716e-05 9.9997187e-01]
[1.3517558e-06 9.9999869e-01]
[1.6466738e-06 9.9999833e-01]
[0.00602899 0.993971  ]
[0.02560974 0.9743903 ]
[0.00119707 0.9988029 ]
[0.5406058  0.45939422]
[2.2303079e-06 9.9999774e-01]
[0.00685151 0.9931485 ]
[0.22444601 0.775554  ]
[0.48044956 0.51955044]
[0.00101137 0.998988